In [4]:
# 기본라이브러리 불러오기

import pandas as pd
import seaborn as sns

df = sns.load_dataset('titanic')

In [6]:
rdf = df.drop(['deck', 'embark_town'], axis = 1)

rdf = rdf.dropna(subset = ['age'], how = 'any' , axis = 0)

most_freq = rdf['embarked'].value_counts(dropna = True).idxmax ()
rdf['embarked'].fillna(most_freq, inplace = True)

In [7]:
# 분석에 활용할 열 (속성)을 선택
ndf = rdf[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'embarked']]
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked
0,0,3,male,22.0,1,0,S
1,1,1,female,38.0,1,0,C
2,1,3,female,26.0,0,0,S
3,1,1,female,35.0,1,0,S
4,0,3,male,35.0,0,0,S


In [8]:
onehot_sex = pd.get_dummies(ndf['sex'])
ndf = pd.concat([ndf, onehot_sex], axis = 1)
onehot_embarked = pd.get_dummies(ndf['embarked'], prefix = 'town')
ndf = pd.concat([ndf, onehot_embarked], axis = 1)
ndf.drop(['sex','embarked'], axis = 1, inplace = True)

ndf.head()

,survived,pclass,age,sibsp,parch,female,male,town_C,town_Q,town_S
0,0,3,22.0,1,0,0,1,0,0,1
1,1,1,38.0,1,0,1,0,1,0,0
2,1,3,26.0,0,0,1,0,0,0,1
3,1,1,35.0,1,0,1,0,0,0,1
4,0,3,35.0,0,0,0,1,0,0,1


In [10]:
# 속성( 변수 ) 선택

X=ndf[['pclass', 'age', 'sibsp', 'parch', 'female', 'male',
       'town_C','town_Q','town_S']]
y = ndf['survived']

from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 10)

In [12]:
print('train data 개수: ', X_train.shape)
print('test data 개수: ', X_test.shape)

train data 개수:  (499, 9)
test data 개수:  (215, 9)


In [13]:
from sklearn import svm

svm_model = svm.SVC()

svm_model.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [14]:
y_hat = svm_model.predict(X_test)

In [18]:
import pandas as pd
df = pd.DataFrame({'y_hat' : y_hat , 'y' : y_test})
df['차이'] = df['y_hat'] == df['y']

In [19]:
df.head()

,y_hat,y,차이
728,0,0,True
555,0,0,True
426,1,1,True
278,0,0,True
617,0,0,True


In [20]:
print('오답 수: ', len(df)- sum(df['차이'])) # True 는 1 이니까

오답 수:  40


In [21]:
from sklearn import metrics
svm_matrix = metrics.confusion_matrix(y_test, y_hat)
svm_matrix

array([[120,   5],
       [ 35,  55]])

In [23]:
svm_report = metrics.classification_report(y_test, y_hat)
print(svm_report)

              precision    recall  f1-score   support

           0       0.77      0.96      0.86       125
           1       0.92      0.61      0.73        90

    accuracy                           0.81       215
   macro avg       0.85      0.79      0.80       215
weighted avg       0.83      0.81      0.81       215

